# NK landscape

- Calculates all possible values in advance **Can take really long for high N**
- Best use: 
    - Calculate needed amount of environments and save them in a file
    - While working only load them and use the dictionary


Changes (if landscape is not reused):
- If payoff needs to be scaled: Implement Branch and Bound algorithm to find optimum & minumum
- Calculate fitness every time (slow)

Bugs:
- Dependencies are not random. The same components are connected every time (np.roll command)


Implemented based on theory description in: https://link.springer.com/article/10.1186/s41469-018-0039-0 

In [15]:
import numpy as np
from itertools import product

In [16]:
class NK_landscape():
    
    def __init__(self, N=10, K=5, C = None, dependencies = None):
        """
        Calculates a dictionary with values for every possible state. 
        Payoff is scaled form 0-1
        C: Table with values for every combination of connected components 
        fitness_dict: keys are the base10 converted states, values the fitness payoff 
        """
        assert N >= K+1
        
        self.N = N
        self.K = K
        
        if C is None or dependencies is None:
            self.initialize_C()
            self.initialize_dependencies()
        else:
            self.dependencies = dependencies
            self.C = C

        
        self.combinations = np.array(list(product([0,1], repeat=K+1)))
        self.max_value = None
        self.min_value = None
        
        self.fitness_dict = {}
        self.init_values()
        
    
    def initialize_dependencies(self):
        """[:,0] is connected with [:,1] --> [0,1] would mean node 0 has fluence on node 1"""
        depend = np.zeros((self.N, self.K+1))
        x = np.arange(self.N)
        depend[:,0] = x
        for _ in range(self.K):
            depend[:,_+1] = np.roll(x,_+1)  # TODO: Create random connections between dimensions

        self.dependencies = depend.astype(int)
    
    def initialize_C(self):
        self.C = np.random.random((self.N,2**(self.K+1))).round(1)

    def f(self, state):
        """Checks all connected dimensions for the combination they are currently in and adds the value"""
        assert len(state) == self.N
        solution = np.empty(len(state))
        for i in range(len(self.dependencies)): 
            solution[i] = self.C[i, int("".join(map(str, state[self.dependencies[i]])), 2)]
        return round(solution.mean(), 2)
    
    def init_values(self):
        """Creates a dictionary mapping all states to their payoff"""
        max_val = 0
        min_val = 2
        for sol in np.array(list(product([0,1], repeat=self.N))):
            key = int("".join(map(str, sol)), 2)
            value = self.f(sol)
            self.fitness_dict[key] = value
          
            if max_val < value:
                max_val = value
            if min_val > value:
                min_val = value
        self.max_value = max_val
        self.min_value = min_val
        
    
    def get_scaled_fitness_new_calculation(self, state):
        """Use if min max are known, but not all values are generated"""
        assert self.max_value != None
        return np.round((self.f(state) - self.min_value) / (self.max_value - self.min_value), 2)
    
    def get_scaled_fitness(self, state):
        key = int("".join(map(str, state)), 2)
        value = self.fitness_dict[key]
        return np.round((value - self.min_value) / (self.max_value - self.min_value), 2)

In [17]:
class NK_landscape_loaded(NK_landscape):
    
    def __init__(self, N, K, fitness_dict):
        assert N >= K+1
        
        self.N = N
        self.K = K

        
        self.max_value = None
        self.min_value = None
        
        self.fitness_dict = fitness_dict
        self.init_min_max()
                
    
    def f(self, state):
        raise NotImplementedError("Only dictionary in loaded nk landscape")
        
    
    def init_min_max(self):
        self.max_value = max(self.fitness_dict.values())
        self.min_value = min(self.fitness_dict.values())
        

### Calculate and save performance table

In [18]:
import pickle 

In [22]:
N = 10
K = 5

n_envs = 20

landscapes = {}

for seed in np.arange(n_envs):
    rng = np.random.default_rng(seed)
    env = NK_landscape(N, K)
    landscapes[seed] = env.fitness_dict
    if seed % 10 == 0:
        print(seed)

file = f'N{N}K{K}_{n_envs}.pkl' 

with open(file, 'wb') as f:
    pickle.dump(landscapes, f)


0
1
2
3
4


In [20]:
with open(file, 'rb') as f:
    data = pickle.load(f)

In [21]:
# Example load
nk = NK_landscape_loaded(N, K, data[0])
state = np.round(rng.random(env.N)).astype(int)
nk.get_scaled_fitness(state)

0.58